In [1]:
#Importamos librerias que vamos a usar
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
import matplotlib.pyplot as plt

# Preprocesado y modelado
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import export_graphviz
from sklearn.tree import export_text
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Configuración warnings
import warnings
warnings.filterwarnings('once')

In [4]:
# Cargar los datasets
train_df = pd.read_csv('/content/drive/MyDrive/data science/bitcoin.csv', sep=',', engine='python')
#test_df = pd.read_csv('/content/drive/MyDrive/data science/test.csv')

print("Dimensiones del set de entrenamiento:", train_df.shape) # Imprimimos para ver
#print("Dimensiones del set de prueba:", test_df.shape) # Imprimimos para ver

# Vistazo rápido a los primeros registros del set de entrenamiento
print("\nPrimeras filas del set de entrenamiento:")
train_df.head() # Imprimimos para ver

Dimensiones del set de entrenamiento: (2500, 9)

Primeras filas del set de entrenamiento:


<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


,timeOpen,timeClose,timeHigh,timeLow,priceOpen,priceHigh,priceLow,priceClose,volume
0,1752840000000,1752926399999,1752861780000,1752914340000,119284.111791,120851.917826,116925.981240,118003.224496,7.794580e+10
1,1752753600000,1752839999999,1752830760000,1752798660000,118738.506039,120999.607158,117508.216047,119289.842192,7.236384e+10
2,1752667200000,1752753599999,1752743040000,1752676680000,117777.188995,120065.518667,117064.821564,118738.506039,7.216203e+10
3,1752580800000,1752667199999,1752582540000,1752634680000,119853.848476,119935.560408,115765.686727,117777.188995,9.832166e+10
4,1752494400000,1752580799999,1752522000000,1752499920000,119115.787510,123091.612801,118959.196786,119849.705720,1.817464e+11


##A)Debemos describir brevemente la cantidad de valores faltantes para cada una de las variables de la base de datos.

In [1]:
# Calcular valores faltantes para el set de entrenamiento
missing_train = train_df.isnull().sum()
missing_train = missing_train[missing_train > 0] # Filtrar solo las columnas con 0 par a que sea mas limpio
print("Valores faltantes en train.csv:") # Imprimimos para ver
print(missing_train)
print("-" * 30) # Separamos con _

# Calcular valores faltantes para el set de prueba
missing_test = test_df.isnull().sum()
missing_test = missing_test[missing_test > 0] # Filtrar solo las columnas con 0
print("Valores faltantes en test.csv:") # Imprimimos para ver
print(missing_test)

NameError: name 'train_df' is not defined

## b) Completar aquellas variables que se encuentran faltantes para las bases de train (“Age”,”fare”, “Cabin”) y test ( “Age”, “Cabin”, “embarked”).

In [ ]:
# --- Imputación para el set de ENTRENAMIENTO (train.csv) ---

# Rellenar 'Age' con la mediana
median_age_train = train_df['Age'].median()
train_df['Age'] = train_df['Age'].fillna(median_age_train)

# Rellenar 'Embarked' con la moda
mode_Embarked_train = train_df['Embarked'].mode()[0]
train_df['Embarked'] = train_df['Embarked'].fillna(mode_Embarked_train)

# Rellenar 'Cabin' con 'U' solo para que de 0 porque despues lo borro
train_df['Cabin'] = train_df['Cabin'].fillna('U')


# --- Imputación para el set de PRUEBA (test.csv) ---

# Rellenar 'Age' con la mediana
median_age_test = test_df['Age'].median()
test_df['Age'] = test_df['Age'].fillna(median_age_test)

# Rellenar 'Fare' con la mediana
median_Fare_test = test_df['Fare'].median()
test_df['Fare'] = test_df['Fare'].fillna(median_Fare_test)

# Rellenar 'Cabin' con 'U' solo para que de 0 porque despues lo borro
test_df['Cabin'] = test_df['Cabin'].fillna('U')

# --- Verificación --- Imprimimos para ver
# Debe quedar en 0

print("\nValores faltantes en train.csv después de la imputación (entrenamiento):")
print(train_df.isnull().sum())
print("\nValores faltantes en test.csv después de la imputación (prueba):")
print(test_df.isnull().sum())


Valores faltantes en train.csv después de la imputación (entrenamiento):
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Valores faltantes en test.csv después de la imputación (prueba):
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


## c) Ahora, debemos ajustar nuestro primer modelo de árbol de decisión.

In [ ]:
# Convertir variables categóricas a numéricas
#train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
#test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

# Eliminar columnas que no usaremos en el modelo inicial
#train_df_model = train_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
#test_df_model = test_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Convertir variables categóricas a numéricas usando One-Hot Encoding
#train_df_processed = pd.get_dummies(train_df_model, columns=['Embarked'], drop_first=True)
#test_df_processed = pd.get_dummies(test_df_model, columns=['Embarked'], drop_first=True)

# Definir nuestras variables (X) y nuestro objetivo (y)
X = train_df_processed.drop('Survived', axis=1)
y = train_df_processed['Survived']

# El set de prueba final será test_df_processed
X_final_test = test_df_processed

print("Variables listas para el modelo:") # Imprimimos para ver
display(X.head())

Variables listas para el modelo:


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,0,22.0,1,0,7.2500,False,True
1,1,1,38.0,1,0,71.2833,False,False
2,3,1,26.0,0,0,7.9250,False,True
3,1,1,35.0,1,0,53.1000,False,True
4,3,0,35.0,0,0,8.0500,False,True


In [5]:
# 1. Dividir el set de entrenamiento en entrenamiento y validación (70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

NameError: name 'X' is not defined

In [6]:
modelo = DecisionTreeClassifier(
            max_depth         = 5,
            criterion         = 'gini',
            random_state      = 42
          )
modelo.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [7]:
# 4. Predesimos sobre el conjunto de validación
y_pred_val = modelo.predict(X_val)


NameError: name 'X_val' is not defined

In [ ]:
#Vamos a mostrar el acurasy

In [ ]:
# Calcular el accuracy
accuracy_val = accuracy_score(y_val, y_pred_val)

print(f"Accuracy en el conjunto de validación: {accuracy_val:.4f}")

Accuracy en el conjunto de validación: 0.7948
